In [7]:
from google.cloud import bigquery
SERVICE_ACCOUNT_JSON=r'C:\Users\saikr\Downloads\modular-asset-332406-b566e60f8085.json'

client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_JSON)

In [2]:
dataset_id="modular-asset-332406.assign1"
dataset=bigquery.Dataset(dataset_id)
dataset.location="US"
dataset.description="python dataset"

dataset_ref= client.create_dataset(dataset,timeout=30)
print("{} is created in {}".format(dataset_ref.dataset_id,client.project))

assign1 is created in modular-asset-332406


In [3]:
tab_id="modular-asset-332406.assign1.clientdetails"
job_config=bigquery.LoadJobConfig(autodetect=True,
                                 source_format=bigquery.SourceFormat.CSV,
                                 skip_leading_rows=1
                                 )
file_path=r"C:\Users\saikr\Downloads\completedclient.csv"
source_file=open(file_path,"rb")
job=client.load_table_from_file(source_file,tab_id,job_config=job_config)
job.result()
table=client.get_table(tab_id)
print("loaded {} rows in {}".format(table.num_rows, tab_id))

loaded 5369 rows in modular-asset-332406.assign1.clientdetails


In [4]:
tab_id="modular-asset-332406.assign1.accdetails"
job_config=bigquery.LoadJobConfig(autodetect=True,
                                 source_format=bigquery.SourceFormat.CSV)
uri="gs://sai_1999/completedacct.csv"
load_job=client.load_table_from_uri(uri,
                                   tab_id,
                                   job_config=job_config
                                   )
load_job.result()
table=client.get_table(tab_id)
print("loaded {} rows in {}".format(table.num_rows, tab_id))

loaded 4500 rows in modular-asset-332406.assign1.accdetails


In [3]:
tab_id="modular-asset-332406.assign1.loandata"
job_config=bigquery.LoadJobConfig(autodetect=True,
                                 source_format=bigquery.SourceFormat.CSV,
                                 skip_leading_rows=1,
                                 clustering_fields=['duration','purpose'])
file_path=r"C:/Users/saikr/Downloads/completedloan.csv"
source_file=open(file_path,"rb")
job=client.load_table_from_file(source_file,tab_id,job_config=job_config)
job.result()
table=client.get_table(tab_id)
print("loaded {} rows in {}".format(table.num_rows, tab_id))

loaded 682 rows in modular-asset-332406.assign1.loandata


In [4]:
tab_id="modular-asset-332406.assign1.districtdetails"
table=client.get_table(tab_id)
print("loaded {} rows in {}".format(table.num_rows, tab_id))

loaded 77 rows in modular-asset-332406.assign1.districtdetails


In [8]:
import pandas as pd
query="""SELECT * FROM `modular-asset-332406.assign1.loandata`
        WHERE amount>10000"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,loan_id,account_id,amount,duration,payments,status,year,month,day,fulldate,location,purpose
0,L00006876,A00009236,86616,12,7218,A,2013,12,21,2013-12-21,1,home
1,L00005314,A00001787,96396,12,8033,B,2013,7,5,2013-07-05,64,home
2,L00007284,A00011265,52788,12,4399,A,2013,9,15,2013-09-15,1,home_improvement
3,L00006820,A00009034,38148,12,3179,A,2013,12,16,2013-12-16,1,debt_consolidation
4,L00006456,A00007123,47016,12,3918,A,2013,12,9,2013-12-09,1,home
...,...,...,...,...,...,...,...,...,...,...,...,...
673,L00007295,A00011328,280440,60,4674,C,2018,7,18,2018-07-18,1,home
674,L00007200,A00010799,31140,60,519,C,2018,7,11,2018-07-11,1,debt_consolidation
675,L00006861,A00009179,215040,60,3584,C,2018,2,28,2018-02-28,1,home
676,L00006568,A00007713,220440,60,3674,C,2018,2,25,2018-02-25,1,home


In [10]:
#total amount bank has given loan
query="""SELECT SUM(amount) as TOTAL_LOAN FROM `modular-asset-332406.assign1.loandata`"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,TOTAL_LOAN
0,103261740


In [11]:
#group by type of loans
query="""SELECT purpose,sum(amount) as total_loan FROM `modular-asset-332406.assign1.loandata`
         GROUP BY purpose
         ORDER BY sum(amount)"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,purpose,total_loan
0,car,500700
1,home_improvement,1874136
2,debt_consolidation,2992644
3,home,97894260


In [12]:
#group by type of loans
query="""SELECT purpose,sum(payments) as monthly_loan FROM `modular-asset-332406.assign1.loandata`
         GROUP BY purpose
         ORDER BY sum(payments)"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,purpose,monthly_loan
0,car,26089
1,home_improvement,111074
2,debt_consolidation,166915
3,home,2553955


In [15]:
tab_id="modular-asset-332406.assign1.fulldetails"
job_config=bigquery.LoadJobConfig(autodetect=True,
                                 source_format=bigquery.SourceFormat.CSV,
                                 skip_leading_rows=1
                                 )
file_path=r"C:/Users/saikr/Downloads/completeddisposition.csv"
source_file=open(file_path,"rb")
job=client.load_table_from_file(source_file,tab_id,job_config=job_config)
job.result()
table=client.get_table(tab_id)
print("loaded {} rows in {}".format(table.num_rows, tab_id))

loaded 5369 rows in modular-asset-332406.assign1.fulldetails


In [16]:
#joins
#find out the names of customers who took loan greater than 50000
query="""WITH first_join AS(
         SELECT loan_id,a1.account_id,amount,client_id FROM `modular-asset-332406.assign1.loandata` AS a1
         LEFT JOIN `modular-asset-332406.assign1.fulldetails` AS a2
         ON a1.account_id=a2.account_id
         WHERE a2.type="Owner")
         
         SELECT a1.client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount FROM first_join AS a1
         LEFT JOIN `modular-asset-332406.assign1.clientdetails` AS a2
         ON a1.client_id=a2.client_id
         WHERE amount>=50000
         ORDER BY amount"""
df=pd.read_gbq(query,dialect='standard',)
table_id="modular-asset-332406.py_dset.newtable"
df.to_gbq(table_id)

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1005.59it/s]


In [17]:
df

,client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount
0,C00011123,A00009031,Male,52,Connor,Camden,Carlson,907-726-8367,connor.carlson@gmail.com,L00006819,50112
1,C00003414,A00002824,Female,52,Janelle,Jaqueline,Blackmon,767-784-7821,janelle.blackmon@gmail.com,L00005549,50460
2,C00001300,A00001083,Male,20,Jaxen,Edward,Henson,491-898-2540,jaxen.henson@gmail.com,L00005172,50976
3,C00005203,A00004320,Female,54,Kayleigh,Quinn,Gonzalez,508-650-6448,kayleigh.gonzalez@outlook.com,L00005856,51360
4,C00005224,A00004337,Male,50,Thor,Ruger,Arthur,586-243-4486,thor.arthur@gmail.com,L00005861,51408
...,...,...,...,...,...,...,...,...,...,...,...
551,C00003546,A00002936,Male,28,Fisher,Kelvin,Underwood,523-896-8322,fisher.underwood@hotmail.com,L00005569,504000
552,C00000981,A00000817,Male,39,Arlo,Joshua,Cervantes,457-226-6112,arlo.cervantes@gmail.com,L00005132,538500
553,C00002823,A00002335,Female,53,Natalie,Karli,Fish,865-444-1244,natalie.fish@hotmail.com,L00005447,541200
554,C00010997,A00008926,Female,27,Harmony,Sloane,Gonzalez,212-283-1340,harmony.gonzalez@outlook.com,L00006791,566640


In [20]:
#create views based on type of loan and find their name,mail id,phonenumbers
query="""WITH first_join AS(
         SELECT loan_id,a1.account_id,amount,client_id FROM `modular-asset-332406.assign1.loandata` AS a1
         LEFT JOIN `modular-asset-332406.assign1.fulldetails` AS a2
         ON a1.account_id=a2.account_id
         WHERE a2.type="Owner" AND a1.purpose="car")
         
         SELECT a1.client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount FROM first_join AS a1
         LEFT JOIN `modular-asset-332406.assign1.clientdetails` AS a2
         ON a1.client_id=a2.client_id"""
view_id="modular-asset-332406.py_dset.view_car"
view=bigquery.Table(view_id)
view.view_query=query
df=client.create_table(view)
df

Table(TableReference(DatasetReference('modular-asset-332406', 'py_dset'), 'view_car'))

In [21]:
#create views based on type of loan and find their name,mail id,phonenumbers
query="""WITH first_join AS(
         SELECT loan_id,a1.account_id,amount,client_id FROM `modular-asset-332406.assign1.loandata` AS a1
         LEFT JOIN `modular-asset-332406.assign1.fulldetails` AS a2
         ON a1.account_id=a2.account_id
         WHERE a2.type="Owner" AND a1.purpose="home")
         
         SELECT a1.client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount FROM first_join AS a1
         LEFT JOIN `modular-asset-332406.assign1.clientdetails` AS a2
         ON a1.client_id=a2.client_id"""
view_id="modular-asset-332406.py_dset.view_home1"
view=bigquery.Table(view_id)
view.view_query=query
df=client.create_table(view)
df

Table(TableReference(DatasetReference('modular-asset-332406', 'py_dset'), 'view_home1'))

In [22]:
#create views based on type of loan and find their name,mail id,phonenumbers
query="""WITH first_join AS(
         SELECT loan_id,a1.account_id,amount,client_id FROM `modular-asset-332406.assign1.loandata` AS a1
         LEFT JOIN `modular-asset-332406.assign1.fulldetails` AS a2
         ON a1.account_id=a2.account_id
         WHERE a2.type="Owner" AND a1.purpose="debt_consolidation")
         
         SELECT a1.client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount FROM first_join AS a1
         LEFT JOIN `modular-asset-332406.assign1.clientdetails` AS a2
         ON a1.client_id=a2.client_id"""
view_id="modular-asset-332406.py_dset.view_debt_consolidation"
view=bigquery.Table(view_id)
view.view_query=query
df=client.create_table(view)
df

Table(TableReference(DatasetReference('modular-asset-332406', 'py_dset'), 'view_debt_consolidation'))

In [27]:
#lets say we need to verify all address of people who took loan for debt_consolidation so the verification team needs names,address,district code,hone numbers,zipcode etc..
query="""WITH  first_join AS(
         SELECT loan_id,a1.account_id,amount,client_id FROM `modular-asset-332406.assign1.loandata` AS a1
         LEFT JOIN `modular-asset-332406.assign1.fulldetails` AS a2
         ON a1.account_id=a2.account_id
         WHERE a2.type="Owner" AND a1.purpose="debt_consolidation" ),
         
         second_join AS(
         SELECT a1.client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount,zipcode,district_id FROM first_join AS a1
         LEFT JOIN `modular-asset-332406.assign1.clientdetails` AS a2
         ON a1.client_id=a2.client_id)
         
         SELECT * FROM `modular-asset-332406.assign1.districtdetails` AS a1
         RIGHT JOIN second_join as a2
         ON a1.district_id=a2.district_id"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,district_id,city,state_name,state_abbrev,region,division,client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount,zipcode,district_id_1
0,1,New York City,New York,NY,Northeast,Middle Atlantic,C00011126,A00009034,Female,35,Hayden,Christina,Perez,212-895-0493,hayden.perez@outlook.com,L00006820,38148,10026,1
1,55,Lowell,Massachusetts,MA,Northeast,New England,C00012754,A00010364,Male,39,Weston,Jaxon,Guerrero,978-152-8213,weston.guerrero@gmail.com,L00007121,21924,1850,55
2,16,Baltimore,Maryland,MD,South,South Atlantic,C00000976,A00000813,Female,44,Viviana,Anaya,Wolf,757-250-2980,viviana.wolf@outlook.com,L00005130,24312,45525,16
3,38,Bridgeport,Connecticut,CT,Northeast,New England,C00004428,A00003674,Male,45,Wallace,Camryn,Pollard,334-496-9884,wallace.pollard@gmail.com,L00005723,23628,46040,38
4,27,San Antonio,Texas,TX,South,West South Central,C00009915,A00008027,Male,32,Michael,Colton,Daniels,380-168-9140,michael.daniels@gmail.com,L00006642,77640,49848,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,45,Cheyenne,Wyoming,WY,West,Mountain,C00001300,A00001083,Male,20,Jaxen,Edward,Henson,491-898-2540,jaxen.henson@gmail.com,L00005172,50976,48654,45
71,27,San Antonio,Texas,TX,South,West South Central,C00007186,A00005944,Female,55,Sadie,Summer,Perez,962-272-8729,sadie.perez@gmail.com,L00006215,78720,48455,27
72,38,Bridgeport,Connecticut,CT,Northeast,New England,C00008096,A00006706,Female,20,Lydia,Lola,Anderson,520-639-3102,lydia.anderson@gmail.com,L00006357,64860,47149,38
73,70,Los Angeles,California,CA,West,Pacific,C00001130,A00000938,Female,59,Lilian,Kelly,Pratt,847-934-4836,lilian.pratt@outlook.com,L00005147,22260,47342,70


In [29]:
query="""WITH  first_join AS(
         SELECT loan_id,a1.account_id,amount,client_id FROM `modular-asset-332406.assign1.loandata` AS a1
         LEFT JOIN `modular-asset-332406.assign1.fulldetails` AS a2
         ON a1.account_id=a2.account_id
         WHERE a2.type="Owner" AND a1.purpose="debt_consolidation" ),
         
         second_join AS(
         SELECT a1.client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount,zipcode,district_id FROM first_join AS a1
         LEFT JOIN `modular-asset-332406.assign1.clientdetails` AS a2
         ON a1.client_id=a2.client_id)
         
         SELECT count(client_id) AS total_clients,region FROM `modular-asset-332406.assign1.districtdetails` AS a1
         RIGHT JOIN second_join as a2
         ON a1.district_id=a2.district_id
         GROUP BY region"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,total_clients,region
0,42,Northeast
1,17,South
2,14,West
3,2,Midwest


In [32]:
query="""SELECT amount,duration,payments,
         CASE WHEN (payments*duration=amount) THEN "NO ERROR"
              WHEN (payments*duration=amount) THEN "ERROR IN AMOUNT,PLEASE CHECK"
         END AS CHECK_LOAN
         FROM `modular-asset-332406.assign1.loandata`"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,amount,duration,payments,CHECK_LOAN
0,86616,12,7218,NO ERROR
1,96396,12,8033,NO ERROR
2,52788,12,4399,NO ERROR
3,38148,12,3179,NO ERROR
4,47016,12,3918,NO ERROR
...,...,...,...,...
677,280440,60,4674,NO ERROR
678,31140,60,519,NO ERROR
679,215040,60,3584,NO ERROR
680,220440,60,3674,NO ERROR


In [33]:
query="""SELECT CAST('5' AS INTEGER)"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,f0_
0,5


In [34]:
query="""SELECT CAST(parseddate AS TIMESTAMP) AS time_stamp FROM `modular-asset-332406.assign1.accdetails`
         """
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,time_stamp
0,2013-01-01 00:00:00+00:00
1,2013-01-01 00:00:00+00:00
2,2013-01-01 00:00:00+00:00
3,2013-01-01 00:00:00+00:00
4,2013-01-02 00:00:00+00:00
...,...
4495,2017-12-28 00:00:00+00:00
4496,2017-12-28 00:00:00+00:00
4497,2017-12-28 00:00:00+00:00
4498,2017-12-29 00:00:00+00:00


In [40]:
query="""SELECT CHAR_LENGTH(CAST(payments AS STRING)) AS loan_figure FROM `modular-asset-332406.assign1.loandata`
         ORDER BY loan_figure"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,loan_figure
0,3
1,3
2,3
3,3
4,3
...,...
677,4
678,4
679,4
680,4


In [43]:
query="""SELECT a1.client_id AS owner_id,a2.client_id AS user_id,a1.account_id FROM `modular-asset-332406.assign1.fulldetails` AS a1
         JOIN `modular-asset-332406.assign1.fulldetails` AS a2
         ON a1.account_id=a2.account_id
         WHERE a1.type="Owner" AND a2.type="User" """
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,owner_id,user_id,account_id
0,C00000002,C00000003,A00000002
1,C00000004,C00000005,A00000003
2,C00000010,C00000011,A00000008
3,C00000015,C00000016,A00000012
4,C00000017,C00000018,A00000013
...,...,...,...
864,C00013447,C00013448,A00010940
865,C00013468,C00013469,A00010954
866,C00013657,C00013658,A00011111
867,C00013750,C00013751,A00011186


In [49]:
query="""WITH  first_join AS(
         SELECT loan_id,a1.account_id,amount,client_id,purpose,duration FROM `modular-asset-332406.assign1.loandata` AS a1
         LEFT JOIN `modular-asset-332406.assign1.fulldetails` AS a2
         ON a1.account_id=a2.account_id
         WHERE a2.type="Owner"),
         
         second_join AS(
         SELECT a1.client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount,zipcode,district_id FROM first_join AS a1
         LEFT JOIN `modular-asset-332406.assign1.clientdetails` AS a2
         ON a1.client_id=a2.client_id)
         
         SELECT * FROM second_join
         
         """
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount,zipcode,district_id
0,C00011372,A00009236,Male,38,Hudson,Cole,Franklin,976-117-5351,hudson.franklin@gmail.com,L00006876,86616,48401,11
1,C00002166,A00001787,Female,52,Kaidence,Giavanna,Lancaster,653-502-9864,kaidence.lancaster@gmail.com,L00005314,96396,45376,30
2,C00013845,A00011265,Male,26,Grayson,Nathaniel,Hale,464-832-7572,grayson.hale@outlook.com,L00007284,52788,46415,15
3,C00011126,A00009034,Female,35,Hayden,Christina,Perez,212-895-0493,hayden.perez@outlook.com,L00006820,38148,10026,1
4,C00008598,A00007123,Female,42,Vivian,Jayla,Lewis,539-422-5777,vivian.lewis@gmail.com,L00006456,47016,42508,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,C00013924,A00011328,Female,47,Elaina,Cataleya,Hernandez,451-056-4090,elaina.hernandez@outlook.com,L00007295,280440,43854,54
678,C00013276,A00010799,Male,23,Jayden,Parker,Shelton,914-740-8456,jayden.shelton@outlook.com,L00007200,31140,10702,76
679,C00011304,A00009179,Female,39,Daniela,Heidi,Pena,331-759-5512,daniela.pena@outlook.com,L00006861,215040,49269,28
680,C00009546,A00007713,Female,44,Mary,Kali,Rodriguez,716-746-3275,mary.rodriguez@hotmail.com,L00006568,220440,14280,50


In [47]:
tab_id="modular-asset-332406.assign1.finaltable"
job_config=bigquery.QueryJobConfig()
job_config.destination=tab_id
query_job=client.query(query,job_config)
query_job.result()



In [50]:
tab_id="modular-asset-332406.assign1.finaltable"
destination_uri="gs://sai_1999/bank_details.csv"
tab_ref=bigquery.Table(tab_id)
extract_job=client.extract_table(tab_ref,
                                destination_uri)
extract_job.result()

ExtractJob<project=modular-asset-332406, location=US, id=72c6719e-796c-4d04-8ea3-17159dd0ff64>

In [52]:
query="""SELECT * FROM `modular-asset-332406.assign1.finaltable`
         FOR SYSTEM TIME AS OF TIMESTAMP_ADD(CURRENT_TIMESTAMP(), INTERVAL -1 MINUTE)"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,client_id,account_id,sex,age,first,middle,last,phone,email,loan_id,amount,zipcode,district_id
0,C00012144,A00009869,Male,19,Blake,Cameron,Fowler,804-377-3995,blake.fowler9@hotmail.com,L00007004,331560,45240,2
1,C00005606,A00004650,Male,19,Jacob,Siddharth,Lewis,808-626-9008,jacob.lewis@hotmail.com,L00005938,402000,47037,4
2,C00008359,A00006924,Male,20,Carlos,Hiram,Tucker,302-865-0282,carlos.tucker@outlook.com,L00006403,194880,44791,74
3,C00012695,A00010320,Male,20,Carlos,Hunter,Barrett,595-517-1902,carlos.barrett@gmail.com,L00007104,259740,40103,68
4,C00002277,A00001876,Male,20,Chaz,Cesar,Byrne,212-826-1832,chaz.byrne8@gmail.com,L00005334,209040,10025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,C00008615,A00007136,Female,60,Quinn,Juliet,Walker,561-527-6729,quinn.walker@gmail.com,L00006461,74124,42344,12
678,C00000025,A00000019,Female,60,Elizabeth,Lupita,Taylor,184-276-9486,elizabeth.taylor@gmail.com,L00004961,30276,42325,21
679,C00000383,A00000314,Female,60,Vanessa,Aliyah,Dunn,959-339-4547,vanessa.dunn67@gmail.com,L00005039,66840,48259,15
680,C00006922,A00005724,Female,60,Anna,Presley,Lopez,946-090-8622,anna.lopez@gmail.com,L00006173,232560,43246,14


In [54]:
query="""SELECT * FROM `modular-asset-332406.assign1.INFORMATION_SCHEMA.PARTITIONS`"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,table_catalog,table_schema,table_name,partition_id,total_rows,total_logical_bytes,total_billable_bytes,last_modified_time,storage_tier
0,modular-asset-332406,assign1,accdetails,None,4500,311190,311190,2021-12-18 12:39:14.619000+00:00,ACTIVE
1,modular-asset-332406,assign1,districtdetails,2021121907,77,4489,4489,2021-12-19 07:15:51.935000+00:00,ACTIVE
2,modular-asset-332406,assign1,loandata,None,682,66290,66290,2021-12-19 09:19:26.788000+00:00,ACTIVE
3,modular-asset-332406,assign1,clientdetails,None,5369,998771,998771,2021-12-18 12:38:47.411000+00:00,ACTIVE
4,modular-asset-332406,assign1,fulldetails,None,5369,213891,213891,2021-12-19 12:38:32.586000+00:00,ACTIVE
5,modular-asset-332406,assign1,finaltable,None,682,92935,92935,2021-12-20 06:26:02.440000+00:00,ACTIVE


In [55]:
query="""SELECT * FROM `modular-asset-332406.assign1.__TABLES__`"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,project_id,dataset_id,table_id,creation_time,last_modified_time,row_count,size_bytes,type
0,modular-asset-332406,assign1,accdetails,1639831154619,1639831154619,4500,311190,1
1,modular-asset-332406,assign1,clientdetails,1639831127411,1639831127411,5369,998771,1
2,modular-asset-332406,assign1,districtdetails,1639898151935,1639898151935,77,4489,1
3,modular-asset-332406,assign1,finaltable,1639981562440,1639981562440,682,92935,1
4,modular-asset-332406,assign1,fulldetails,1639917512586,1639917512586,5369,213891,1
5,modular-asset-332406,assign1,loandata,1639905566788,1639905566788,682,66290,1


In [59]:
query="""SELECT TIMESTAMP_MILLIS(creation_time) AS time,dataset_id FROM `modular-asset-332406.assign1.__TABLES__`"""
df=pd.read_gbq(query,project_id='modular-asset-332406',dialect='standard')
df

,time,dataset_id
0,2021-12-18 12:39:14.619000+00:00,assign1
1,2021-12-18 12:38:47.411000+00:00,assign1
2,2021-12-19 07:15:51.935000+00:00,assign1
3,2021-12-20 06:26:02.440000+00:00,assign1
4,2021-12-19 12:38:32.586000+00:00,assign1
5,2021-12-19 09:19:26.788000+00:00,assign1
